In [1]:
import numpy as np
import cvxpy as cp
import random

### Q1 d.2) Solving the sparse recovery problem using CVXPY

- The sparse recovery problem for the fewest-pigments formulation is given by:
$$
\min_{x \in \mathbb{R}^3} \quad \|x\|_0 \\
\text{s.t.} \quad Ax = b, \\
\quad x \ge 0
$$
- However, the above problem is not a convex optimisation problem. CVXPY requires the problem to be convex inorder for it to be solved. Hence, we shall use the $l_1$ relaxation of the sparse recovery problem as a proxy as it is a convex problem. The $l_1$ problem is formulated as:
$$
\min_{x \in \mathbb{R}^3} \quad \|x\|_1 \\
\text{s.t.} \quad Ax = b, \\
\quad x \ge 0
$$

In [2]:
#Initialising the matrix A and vector b
A = np.array([[1,0,1],
              [0,1,1],
              [1,1,2]],dtype=np.float32)
b = np.array([0.2,0.8,1.0])

In [3]:
#Initialising the solution vector x, the constraints, and the objective function
x = cp.Variable(3)
constraints = [A@x == b,x >= 0]
objective = cp.Minimize(cp.norm1(x)) #We are using the L1 relaxation since L0 is not convex and would not follow the expected structure that CVXPY requires

In [4]:
#Setting up the problem to be solved
problem = cp.Problem(objective,constraints)

In [5]:
#Checking if the problem formulation is disciplined (follows the expected structure that CVXPY requires)
print("problem is DCP:", problem.is_dcp())

problem is DCP: True


In [11]:
#Solving the convex optimisation problem and checking if optimality is achieved, and what are the optimal solutions, and optimal objective values
problem.solve() 
print("status:", problem.status)
print("optimal value of the objective:", problem.value)
print("optimal value of x:", x.value)
print("Optimal support of x:", np.nonzero(np.round(x.value, 6))[0])
print("Residual norm:", np.linalg.norm(A @ x.value - b))

status: optimal
optimal value of the objective: 0.8000000014092193
optimal value of x: [1.40921960e-09 6.00000001e-01 1.99999999e-01]
Optimal support of x: [1 2]
Residual norm: 2.056349140077291e-15


- CVXPY was able to achieve an optimal solution.
- The optimal value of $x$ is [0,0.6,0.2]. However due to numerical precision and rounding, the values reported by the solver are off by very small amounts, which can be resolved by rounding it off.
- This was one of the solutions when the sparse recovery problem was solved by hand, and is the exact solution that was obtained when the $l_1$ relaxation was solved by hand.
- The support of $x$ is [1,2] (0-indexing in python) or [2,3] (1-indexing as in regular linear algebra notations). 
- Residual norm $||Ax - b||_2$ is very close to zero, indicating an exact solution to the constraint $Ax = b$.

### Q1 e.2) Solving the modified sparse recovery problem using CVXPY

- Since the constraint $Ax = b$ is infeasible, we have replaced it with a relaxed constraint $||Ax - b||_2 \leq t$ and introduced the slack variable $t$ in the objective.
- The modified $l_1$ relaxation problem is now:
$$
\min_{x \in \mathbb{R}^3, t \in \mathbb{R}} \quad \|x\|_1 + t \\
\text{s.t.} \quad ||Ax - b||_2 \leq t, \\
\quad x \ge 0
\quad t \ge 0
$$

In [12]:
#Initialising the matrix A and vector b
A = np.array([[1,0,1],
              [0,1,1],
              [1,1,2]],dtype=np.float32)
b = np.array([0.2,0.8,0.95])

In [13]:
#Initialising the solution vector x, the constraints, and the objective function
x = cp.Variable(3)
t = cp.Variable(nonneg = True) # t is non-negative
constraints = [cp.norm2(A@x - b) <= t,x >= 0]
objective = cp.Minimize(cp.norm1(x) + t) #We are using the L1 relaxation since L0 is not convex and would not follow the expected structure that CVXPY requires

In [14]:
#Setting up the problem to be solved
problem = cp.Problem(objective,constraints)

In [15]:
#Checking if the problem formulation is disciplined (follows the expected structure that CVXPY requires)
print("problem is DCP:", problem.is_dcp())

problem is DCP: True


In [16]:
#Solving the convex optimisation problem and checking if optimality is achieved, and what are the optimal solutions, and optimal objective values
problem.solve() 
print("status:", problem.status)
print("optimal value of objective:", problem.value)
print("optimal value of x:", x.value)
print("Optimal support of x:", np.nonzero(np.round(x.value, 6))[0])
print("Residual norm:", np.linalg.norm(A @ x.value - b))

status: optimal
optimal value of objective: 0.7999999996093653
optimal value of x: [1.28998046e-09 5.50003877e-01 1.99998353e-01]
Optimal support of x: [1 2]
Residual norm: 0.04999777018216895


- CVXPY has attained an optimal solution to the modified problem.
- The optimal value of $x$ is [0,0.55,0.2], but as seen before, the values are off by a very small amount due to floating point precision and rounding.
- The support of $x$ is same as it was for the original problem: [1,2] (0-indexing) or [2,3] (regular indexing).
- The residual norm now is 0.049, which shows that the solver was able to find a solution that minimised residual norm, and at the same time, was able to give a relatively more sparse solution than the original problem.

### Q2 c.1) Solving nuclear norm minimisation using CVXPY

- The rank minimisation problem is formulated as:
$$
\min_{X \in \mathbb{R}^{p\times n}} \; \text{rank}(X) 
\quad \text{s.t.} \quad AX = B
$$
- However, this is not a convex problem and hence, CVXPY cannot solve it directly.
- Instead, to solve the Multi-Experiment system identification problem, we shall use the nuclear norm minimisation problem as a proxy as it is a convex problem.
$$
\min_{X \in \mathbb{R}^{p\times n}} \; \|X\|_* 
\quad \text{s.t.} \quad AX = B
$$

In [ ]:
#Intialising matrices A and B (5x5 matrices)
A = np.array([
    [1.0,0.0,0.0,0.0,0.0],
    [0.8,1.0,0.0,0.0,0.0],
    [-0.2,0.8,1.0,0.0,0.0],
    [0.5,-0.2,0.8,1.0,0.0],
    [0.0,0.5,-0.2,0.8,1.0]
])
B = np.array([
    [1.0, -1.0, 0.0, 0.0, 1.0],
    [2.8, -0.8, 1.0, -1.0, 0.8],
    [2.4, 1.2, 1.8, -1.8, -1.2],
    [2.9, -1.7, 0.6, -0.6, 1.7],
    [0.4, -1.8, -0.7, 0.7, 1.8]
])

In [ ]:
#Intialising the constraints, objective function, and decision variable X
X = cp.Variable((5,5))  #(5x5) Matrix
constraints = [A@X == B]    
objective = cp.Minimize(cp.normNuc(X))  #Nuclear norm of X

In [20]:
#Setting up the problem to be solved
problem = cp.Problem(objective,constraints)

In [21]:
#Checking if the problem formulation is disciplined (follows the expected structure that CVXPY requires)
print("problem is DCP:", problem.is_dcp())

problem is DCP: True


In [24]:
#Solving the convex optimisation problem and checking if optimality is achieved, and what are the optimal solutions, and optimal objective values
problem.solve() 
print("status:", problem.status)
print("optimal value of objective:", problem.value)
print("optimal value of X (without rounding):")
print(X.value)
print("optimal value of X (with rounding):")
print(np.round(X.value,6))

status: optimal
optimal value of objective: 7.928082986664531
optimal value of X (without rounding):
[[ 9.99999971e-01 -9.99999989e-01 -8.61026973e-09  8.61026972e-09
   9.99999989e-01]
 [ 2.00000006e+00 -2.31914014e-08  1.00000002e+00 -1.00000002e+00
   2.31914014e-08]
 [ 9.99999925e-01  1.00000002e+00  9.99999972e-01 -9.99999972e-01
  -1.00000002e+00]
 [ 2.00000011e+00 -2.00000003e+00  3.97572976e-08 -3.97572975e-08
   2.00000003e+00]
 [-2.00000014e+00  5.25433735e-08 -1.00000005e+00  1.00000005e+00
  -5.25433732e-08]]
optimal value of X (with rounding):
[[ 1. -1. -0.  0.  1.]
 [ 2. -0.  1. -1.  0.]
 [ 1.  1.  1. -1. -1.]
 [ 2. -2.  0. -0.  2.]
 [-2.  0. -1.  1. -0.]]


- CVXPY was able to achieve an optimal solution.
- Due to precision and rounding of floating point values, the optimiser obtained looks a bit messy due to very small decimal point variations. After rounding it off to the $6^{th}$ decimal place, we see that the optimiser has a nice structure.
- The optimal value of the objective is 7.928

### Q2 c.2) Solving modified nuclear norm minimisation using CVXPY

- Since the constraint $AX = B$ is now infeasible due to the addition of noise, we have relaxed the constraint as $||AX - B||_F \leq t$, and introduced the slace variable $t$ in the objective.
- The modified Nuclear Norm Minimisation problem is:
$$
\min_{X \in \mathbb{R}^{p\times n},t \in \mathbb{R}} \; \|X\|_* + t 
\quad \text{s.t.} \quad AX = B
\quad t \ge 0
$$

In [136]:
#Initialising matrices A and B (5x5 matrices)
A = np.array([
    [1.0,0.0,0.0,0.0,0.0],
    [0.8,1.0,0.0,0.0,0.0],
    [-0.2,0.8,1.0,0.0,0.0],
    [0.5,-0.2,0.8,1.0,0.0],
    [0.0,0.5,-0.2,0.8,1.0]
])
B = np.array([
    [1.0, -1.0, 0.0, 0.0, 1.0],
    [2.8, -0.8, 1.0, -1.0, 0.8],
    [2.4, 1.2, 1.8, -1.8, -1.2],
    [2.9, -1.7, 0.6, -0.6, 1.7],
    [0.4, -1.8, -0.7, 0.7, 1.8]
])

- We are fixing a seed so that the noise added via $N$ is consistent and reproducible.
- $N \in \mathbb{R}^{5 \times 5}$ such that $N[i,j] \sim \mathcal{N}(0,0.1)$ if $i = j$ and 0 if $i \neq j$

In [137]:
np.random.seed(42) #Fixing seed for reproducibility
diagonal_entries = np.random.normal(loc=0.0, scale=np.sqrt(0.1), size=5) #Initialising N
N = np.diag(diagonal_entries)
print("N =\n", N)

N =
 [[ 0.15707481  0.          0.          0.          0.        ]
 [ 0.         -0.04372301  0.          0.          0.        ]
 [ 0.          0.          0.2048171   0.          0.        ]
 [ 0.          0.          0.          0.48162433  0.        ]
 [ 0.          0.          0.          0.         -0.0740458 ]]


In [138]:
#Adding N to B to get noisy B
B = B + N
print(B)

[[ 1.15707481 -1.          0.          0.          1.        ]
 [ 2.8        -0.84372301  1.         -1.          0.8       ]
 [ 2.4         1.2         2.0048171  -1.8        -1.2       ]
 [ 2.9        -1.7         0.6        -0.11837567  1.7       ]
 [ 0.4        -1.8        -0.7         0.7         1.7259542 ]]


In [139]:
#Intialising decision variable X, slack variable t, constraints, and objective function
X = cp.Variable((5,5))
t = cp.Variable(nonneg = True)
constraints = [cp.norm(A@X - B,"fro") <= t] #Frobenius norm inequality
objective = cp.Minimize(cp.normNuc(X) + t)  #Nuclear norm + t

In [140]:
#Setting up the problem to be solved
problem = cp.Problem(objective,constraints)

In [141]:
#Checking if the problem formulation is disciplined (follows the expected structure that CVXPY requires)
print("problem is DCP:", problem.is_dcp())

problem is DCP: True


In [142]:
#Solving the convex optimisation problem and checking if optimality is achieved, and what are the optimal solutions, and optimal objective values
problem.solve() 
print("status:", problem.status)
print("optimal value of objective:", problem.value)
print("optimal value of X:")
print(X.value)

status: optimal
optimal value of objective: 6.891657524566875
optimal value of X:
[[ 0.96888831 -0.63148695  0.1687221  -0.08882722  0.61809172]
 [ 0.91715889  0.00184829  0.49293127 -0.42477218 -0.00453722]
 [ 1.05285433  0.09826737  0.61928766 -0.54221194 -0.09974634]
 [ 0.62983802 -0.70055562 -0.05149381  0.1070577   0.68701207]
 [-0.09251433 -0.34239755 -0.23988415  0.23726486  0.3369615 ]]


- CVXPY was able to arrive at an optimal solution.
- The optimal value of the objective is 6.892, which is smaller than the original problem's objective value at optimality.
- The values beyond the $6^{th}$ decimal place change during each run due to floating point precision operations and rounding.